<a href="https://colab.research.google.com/github/michaelknorr/CHE5082/blob/main/CHE5082_Parameter_Estimations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Calculate BP and MP by Group Contribution Method & Estimate Vapor Pressure (in atm) by Antoine Equation**

In [32]:
!pip install chetoolbox
import chetoolbox.common as ch
import chetoolbox.separations as che
import numpy as np
bp_group_contributions = {
    "-CH3": 21.98,
    ">CH2": 24.22,
    '>C_ringH2':26.44,
    '>CH-':11.86,
    '>C_ringH-':21.66,
    '>C<':4.5,
    '>C_ring<':11.12,
    '=CH2':16.44,
    '=CH-':27.95,
    '=C_ringH-':28.03,
    '=C<':23.58,
    '=C_ring<':28.19,
    'aaCH*':28.53,
    'aaC-':30.76,
    'aaaC':45.46,
    '=CH':21.71,
    '=C-':32.99,

    '-OH':106.27,
    'Primary-OH': 88.46,
    'Secondary-OH':80.63,
    'Tertiary-OH':69.32,
    'Aromatic-OH':70.48,
    '-O-':25.16,
    '-O_ring-':32.98,
    '-OOH-': 72.92,

    '-CHO':83.38,
    '>CO':71.53,
    '>C_ringO':94.76,
    '-C(O)O-':78.85,
    '-C_ring(O)O_ring-':172.49,
    '-C(O)OH':169.83,
    '-C(O)NH2':230.39,
    'C(O)NH-':225.09,
    'C_ring(O)N_ringH-':246.13,
    '-C(O)N<':142.77,
    '-C_ring(O)N_ring<':180.22,

    '-NH2':61.98,
    'Aromatic-NH2':86.63,
    '>NH':45.28,
    '>N_ringH':65.5,
    '>N-':25.78,
    '>N_ring-':32.77,
    '>NOH':104.87,
    '>NNO':184.68,
    'anN':39.88,
    '=NH':73.4,
    '=N_ring-':43.54,
    '=N_ringN_ringH-':179.43,
    '-N_ring=C_ringN_ringH-':284.16,
    '-N=NNH-':257.29,
    '-N=N-':90.87,
    '-NO':30.91,
    '-NO2':113.99,
    '-CN':119.16,
    'Aromatic-CN':95.43,

    '-F':0.13,
    'Aromatic-F':-7.81,
    '-Cl':34.08,
    'Primary-Cl':62.63,
    'Secondary-Cl':49.41,
    'Tertiary-Cl':36.23,
    'Aromatic-Cl':36.79,
    '-Br':76.28,
    'Aromatic-Br':61.85,
    '-I':111.67,
    'Aromatic-I': 99.93,

    '-SH': 81.71,
    'Aromatic-SH':77.49,
    '-S-':69.42,
    '-Sring-':69.00,
    '>SO':154.50,
    '>SO2':171.58,
    '>CS':106.20,
    '>CringS':179.26,

}


In [47]:
def calculate_boiling_point(groups):
    bp_group_contributions = {
        "-CH3": 21.98, ">CH2": 24.22, '>C_ringH2': 26.44,
        ">CH-": 11.86, '>C_ringH-': 21.66, '>C<': 4.5,
        '>C_ring<': 11.12, '=CH2': 16.44, '=CH-': 27.95,
        '=C_ringH-': 28.03, '=C<': 23.58, '=C_ring<': 28.19,
        'aaCH': 28.53, 'aaC-': 30.76, 'aaaC': 45.46,
        '=CH': 21.71, '=C-': 32.99, '-OH': 106.27,
        'Primary-OH': 88.46, 'Secondary-OH': 80.63,
        'Tertiary-OH': 69.32, 'Aromatic-OH': 70.48,
        '-O-': 25.16, '-O_ring-': 32.98, '-OOH-': 72.92,
        '-CHO': 83.38, '>CO': 71.53, '>C_ringO': 94.76,
        '-C(O)O-': 78.85, '-C_ring(O)O_ring-': 172.49,
        '-C(O)OH': 169.83, '-C(O)NH2': 230.39,
        'C(O)NH-': 225.09, 'C_ring(O)N_ringH-': 246.13,
        '-C(O)N<': 142.77, '-C_ring(O)N_ring<': 180.22,
        '-NH2': 61.98, 'Aromatic-NH2': 86.63,
        '>NH': 45.28, '>N_ringH': 65.5, '>N-': 25.78,
        '>N_ring-': 32.77, '>NOH': 104.87, '>NNO': 184.68,
        'anN': 39.88, '=NH': 73.4, '=N_ring-': 43.54,
        '=N_ringN_ringH-': 179.43, '-N_ring=C_ringN_ringH-': 284.16,
        '-N=NNH-': 257.29, '-N=N-': 90.87, '-NO': 30.91,
        '-NO2': 113.99, '-CN': 119.16, 'Aromatic-CN': 95.43,
        '-F': 0.13, 'Aromatic-F': -7.81, '-Cl': 34.08,
        'Primary-Cl': 62.63, 'Secondary-Cl': 49.41,
        'Tertiary-Cl': 36.23, 'Aromatic-Cl': 36.79,
        '-Br': 76.28, 'Aromatic-Br': 61.85, '-I': 111.67,
        'Aromatic-I': 99.93, '-SH': 81.71, 'Aromatic-SH': 77.49,
        '-S-': 69.42, '-Sring-': 69.00, '>SO': 154.5,
        '>SO2': 171.58, '>CS': 106.2, '>CringS': 179.26,
    }

    # Base boiling point calculation
    Tb = 198.2 + sum(groups.get(group, 0) * bp_group_contributions.get(group, 0) for group in groups)

    # Apply correction
    if Tb < 700:
        Tb_corrected = Tb - 98.84 + 0.5577 * Tb - 0.0007705 * Tb ** 2
    else:
        Tb_corrected = Tb + 282.7 - 0.5209 * Tb

    #Calculate Melting Point
    Tm=0.5839*Tb_corrected

    if Tm>298:
      print("substance is solid @ room temp")
      T=float(input('Please input the temperature at which the vapor pressure is to be calculated (K): '))
      Pvap=np.exp(-(4.4+np.log10(Tb_corrected))*(1.803*(Tb_corrected/T-1)-0.803*np.log(Tb_corrected/Tb))-6.8*(Tm/T-1))

    elif Tm<298 & Tb>298:
      print("substance is liquid @ room temp")
      Kf= 1.06
      print('calculated with assumed value of 1.06: CHECK TABLES FIRST!!!')
      R=1.987 #Cal/mole.K
      T=float(input('Please input the temperature at which the vapor pressure is to be calculated (K): '))
      C=-19+0.19*Tb_corrected
      A= Kf*(8.75+R*np.log(Tb_corrected))
      Pvap=np.exp(((A*(Tb_corrected-C)**2)/(0.97*1.987*Tb_corrected))*(1/(Tb_corrected-C)-1/(T-C)))

    elif Tb<298:
      print("substance is vapor @ room temp")
    else:
      print('figure out substance state @ room temp on your own')
    return Tb, Tb_corrected, Tm, Pvap

In [48]:
groups = {"aaCH": 3, "aaC-": 9, "Aromatic-Cl": 7}  # Example molecule
boiling_point = calculate_boiling_point(groups)
print(boiling_point)


substance is solid @ room temp
Please input the temperature at which the vapor pressure is to be calculated (K): 298
(818.1600000000001, 674.680456, 393.9459182584, 2.5578397728479394e-09)


**Estimate the Octanol-Water partition coefficient (Kow)**

****

In [52]:
def calculate_Kow(groups):
  Kow_group_contributions = {
        "aaC":0.2940, 'aaO':-0.0423, 'aaS':0.40802, 'aaN_fused':-0.0001, 'aaN_5_membered':-0.5262,
        'aaN_6_membered':-0.7324,

        '-CH3':0.5473, '-CH2-':0.4911, '-CH<':0.3614, '>C<':0.2676, 'C (no H)':0.9723,



    }
  Kow_correction_factors = {
        "-CH3": 21.98, ">CH2": 24.22, '>C_ringH2': 26.44,
        ">CH-": 11.86, '>C_ringH-': 21.66, '>C<': 4.5,
        '>C_ring<': 11.12, '=CH2': 16.44, '=CH-': 27.95,
        '=C_ringH-': 28.03, '=C<': 23.58, '=C_ring<': 28.19,
        'aaCH': 28.53, 'aaC-': 30.76, 'aaaC': 45.46,
        '=CH': 21.71, '=C-': 32.99, '-OH': 106.27,
        'Primary-OH': 88.46, 'Secondary-OH': 80.63,
        'Tertiary-OH': 69.32, 'Aromatic-OH': 70.48,
        '-O-': 25.16, '-O_ring-': 32.98, '-OOH-': 72.92,
        '-CHO': 83.38, '>CO': 71.53, '>C_ringO': 94.76,
        '-C(O)O-': 78.85, '-C_ring(O)O_ring-': 172.49,
        '-C(O)OH': 169.83, '-C(O)NH2': 230.39,
        'C(O)NH-': 225.09, 'C_ring(O)N_ringH-': 246.13,
        '-C(O)N<': 142.77, '-C_ring(O)N_ring<': 180.22,
        '-NH2': 61.98, 'Aromatic-NH2': 86.63,
        '>NH': 45.28, '>N_ringH': 65.5, '>N-': 25.78,
        '>N_ring-': 32.77, '>NOH': 104.87, '>NNO': 184.68,
        'anN': 39.88, '=NH': 73.4, '=N_ring-': 43.54,
        '=N_ringN_ringH-': 179.43, '-N_ring=C_ringN_ringH-': 284.16,
        '-N=NNH-': 257.29, '-N=N-': 90.87, '-NO': 30.91,
        '-NO2': 113.99, '-CN': 119.16, 'Aromatic-CN': 95.43,
        '-F': 0.13, 'Aromatic-F': -7.81, '-Cl': 34.08,
        'Primary-Cl': 62.63, 'Secondary-Cl': 49.41,
        'Tertiary-Cl': 36.23, 'Aromatic-Cl': 36.79,
        '-Br': 76.28, 'Aromatic-Br': 61.85, '-I': 111.67,
        'Aromatic-I': 99.93, '-SH': 81.71, 'Aromatic-SH': 77.49,
        '-S-': 69.42, '-Sring-': 69.00, '>SO': 154.5,
        '>SO2': 171.58, '>CS': 106.2, '>CringS': 179.26,
    }

    # Base boiling point calculation
  Kow = 0.229 + sum(groups.get(group, 0) * Kow_group_contributions.get(group, 0) for group in groups)+ sum(groups.get(group, 0) * Kow_correction_factors.get(group, 0) for group in groups)
  return Kow

In [53]:
#logKow=0.229+sum(n*contribution)+(n*correction) //Octanol-Water Partition Coefficient
#logBCF=0.77(Log(Kow))-0.7 + Sum(correction factors) //Bioconcentration Factor
#LogS=0.342-1.03474*np.log10(Kow)-0.0108*(Tm-25) + Sum of Correction Factors //Water Solubility

logKow=0.229+(n*contribution)+(n*correction)
logBCF=0.77*logKow-0.7 + correction_BCF
logS=0.342-1.03474*logKow-0.0108*(Tm-25) + SumofCorrectionFactors

NameError: name 'n' is not defined